In [ ]:
# Homework 2 Part 1 (due 7/07/2024)
# Abby Irish

# Multivariate linear regression with within-sample validation

### Objective
In this project, you will identify relationships between variables via single-variable and multivariable linear regression using the python library `sci-kit learn`. You will practice assessing the model fit within sample and out of sample.

### Step 1
The following code snippet (1) loads the diabetes dataset from the sci-kit learn package, (2) fits a linear regression to the first variable in that data set, (3) calculate a t statistics for the estimated model parameters, (4) calculates the within-sample R2, and (5) plots the data and the model. Review and test the code.

### Step 2
Write code that  fits a linear models for EACH INDEPENDENT VARIABLE in the data set. For each of the resulting models, (1) calculate a t statistic for the estimated model parameters, (2) calculate the within-sample RSS, MSE, RSE, and R^2. Plot the results for the model that has the best quality of fit.

### Step 3
Write code that  fits a multivariate linear model for ALL INDEPENDENT VARIABLE in the data set. (1) calculate an f statistic on the estimated model parameters, (2) calculate the within-sample RSS, MSE, RSE, and R^2.

### Step 4
Test the information content of each variable in the multivariate model by calculating an f statistic with respect to a reduced model. Are the three variables with the highest f statistic also the variables with the best-fitting single-variable models? Explain what may lead to differences in the variable rankings.

### Step 5
Look up how to use the function `train_test_split` from `sklearn.model_selection` split a dataset into a training set and a test set. Repeat steps 2 and 3 on the training set. For each model, also calculate RSS, MSE, RSE, and R2 on the test set. For each linear model, comment on how RSE and R2 change when moving from the training set to the test. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# Read-in the diabetes dataset as a pandas DataFrame
diabetes = datasets.load_diabetes(as_frame=True)


# Get independent variables
X = diabetes.data
print(X)

# Get dependent variable
y = diabetes.target

# Let's look at the data
X.describe()

In [ ]:
# initialize model
model = LinearRegression()

# get variable names from column header in the data frame
var_names = X.columns

# select first variable
var_name1 = var_names[0]

# select data associated with the first variable
x1 = X[var_name1]

# turn that dataframe column into a nx1 numpy array
x1_data = np.array([x1.to_numpy()]).T

# fit model
_ = model.fit(x1_data,y.to_numpy())

# get model predictions for each x value
yHat = model.predict(x1_data)

# get residuals
resid = yHat-y

# get R2 value
R2 = model.score(x1_data,y)
print('R2', R2)

# make a plot
plt.subplot(111)

# plot data
plt.plot(x1, y, marker='x', lw=0, color='blue')

# plot fit
plt.plot(x1, yHat, ls='--', color='red')

In [ ]:
# STEP 2: Simple Linear Regression

# linear models for each independent variable:
from scipy import stats
var_names = X.columns

for i in range(len(var_names)-1):
    model2 = LinearRegression()
    
    var_name = var_names[i]
    print(var_name)
    
    x = X[var_name]
    
    x_data = np.array([x.to_numpy()]).T
    
    _ = model2.fit(x_data,y.to_numpy())
    
    yHat = model2.predict(x_data)
    
    resid = yHat-y

    yBar = np.mean(y)

# calculate t statistic 
    TSTAT = stats.ttest_1samp(x_data, yBar)
    print("tStat: ", TSTAT)
    
# calculate R^2: 
    R2 = model2.score(x_data,y)
    print('R2: ', R2)

# calculate RSS
    RSS = sum(resid*resid)
    print ("RSS: ", RSS)
    
# calculate  MSE 
    MSE = RSS/len(x_data)
    print("MSE: ", MSE)
    
# calculate RSE 
    RSE = np.sqrt(RSS/(len(resid)-2))
    print("RSE: ", RSE)


# Plot for the model with the best fit: 
model3 = LinearRegression()

x2 = X[var_names[2]] # bmi is variable with best fit
print(var_names[2])

x2_data = np.array([x2.to_numpy()]).T

_ = model3.fit(x2_data,y.to_numpy())

yHat = model3.predict(x2_data)

resid = yHat - y

plt.subplot(111)

# plot data
plt.plot(x2, y, marker='x', lw=0, color='blue')

# plot fit
plt.plot(x2, yHat, ls='--', color='red')

In [ ]:
# STEP 3: Multivariate Linear Regression
all_variables = []

for i in var_names:
    col = X[i]
    col_all_variables = col.to_numpy().reshape(-1, 1)

    all_variables.append(col_all_variables)

x_data = np.column_stack(all_variables)
print(x_data.shape)
print(x_data)

multivariateModel = LinearRegression()
_ = multivariateModel.fit(x_data, y.to_numpy())

yHat = multivariateModel.predict(x_data)
multiResid = yHat - y

# calculate RSS 
multiRSS = sum(multiResid ** 2)
print('RSS: ', multiRSS)

# calculate MSE:
multiMSE = mean_squared_error(yHat, y)
print('MSE: ', multiMSE)

# calculate RSE: 
multiRSE = np.sqrt(multiRSS / (len(multiResid) - 2))
print('RSE: ', multiRSE)

# calculate R^2:
multiR2 = multivariateModel.score(x_data, y)
print('R^2: ', multiR2)

# calculate an f-stat:
yBar = np.mean(y)
multiTSS = sum((y - yBar) ** 2)
multiESS = multiTSS - multiRSS
n = len(y)
k = x_data.shape[1]
fstat = (multiESS / k) / (multiRSS / (n - k - 1))
print("f-stat: ", fstat)

In [ ]:
# Step 4: test the information content of each variable:

f_stats = []

for i in range(k): # interate over each variable to create reduced models
    x_data_reduced = np.delete(x_data, i, axis = 1) # remove the i-th variable

    reduced_model = LinearRegression()
    reduced_model.fit(x_data_reduced, y)
    yHat_reduced = reduced_model.predict(x_data_reduced)
    RSS_reduced = np.sum((y - yHat_reduced) ** 2)

    ESS = RSS_reduced - multiRSS
    f_stat = (ESS / 1) / (multiRSS / (n - k - 1))
    f_stats.append(f_stat)

for i, f_stat in enumerate(f_stats): # get both the variable index and the f-stat value 
    print(f"Variable {var_names[i]}: F-statistic = {f_stat}")

top_three_indices = np.argsort(f_stats)[-3:][::-1]
for i in top_three_indices:
    print(f"{var_names[i]}")

single_var_fits = []
for i in range(k):
    single_var_model = LinearRegression()
    single_var_model.fit(x_data[:, [i]], y)

    yHat_single = single_var_model.predict(x_data[:, [i]])
    RSS_single = np.sum((y - yHat_single) ** 2)
    single_var_fits.append(RSS_single)

top_three_single_indices = np.argsort(single_var_fits)[:3]
for i in top_three_single_indices:
    print(f"{var_names[i]}")